In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets

In [2]:
# Load the data into a pandas DataFrame
data = pd.read_csv('TRANSCO.csv') # Replace 'stock_data.csv' with your data file


In [3]:
# Assuming the columns are named 'Date', 'Open', 'Close', 'High', 'Low', 'Volume'
data['Date'] = pd.to_datetime(data['Date']) # Convert 'Date' column to datetime if needed
data = data.sort_values('Date') # Sort the data by date


In [4]:
data['NextDayClose'] = data['Close'].shift(-1)
data['NextDayPrice'] = data['NextDayClose'] - data['Close']
data['NextDayPercentageChange'] = (data['NextDayClose'] - data['Close']) / data['Close'] * 100

In [5]:
# Split the data into training and testing sets
X = data[['Open', 'Close', 'High', 'Low', 'Vol(millions)']].values[:-1]
y = data[['NextDayClose', 'NextDayPrice', 'NextDayPercentageChange']].values[:-1]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [7]:
# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [8]:
# Evaluate the model on the testing set
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

In [9]:
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")


Mean Squared Error: 3.88
Root Mean Squared Error: 1.19


In [10]:
# Forecast tomorrow's close price, price, and percentage change
last_data = data.iloc[-1][['Open', 'Close', 'High', 'Low', 'Vol(millions)']].values
prediction = model.predict([last_data])
next_day_close = prediction[0][0]
next_day_price = prediction[0][1]
next_day_percentage_change = prediction[0][2]

C:\Users\servicedesk1\Anaconda3\lib\site-packages\sklearn\base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)


In [11]:
print(f"Tomorrow's Close Price Prediction: {next_day_close:.2f}")
#print(f"Tomorrow's Price Prediction: {next_day_price:.2f}")
print(f"Tomorrow's Percentage Change Prediction: {next_day_percentage_change:.2f}%")

Tomorrow's Close Price Prediction: 3.27
Tomorrow's Percentage Change Prediction: -4.93%


In [12]:
# Convert the 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Set the 'Date' column as the index
data.set_index('Date', inplace=True)

# Create a filter dropdown for selecting the time interval
interval_dropdown = widgets.Dropdown(
    options=['Hourly', 'Daily', 'Monthly', 'Yearly'],
    value='Daily',
    description='Interval:'
)

# Define a function to update the plot based on the selected time interval
def update_plot(interval):
    if interval == 'Hourly':
        resampled_data = data.resample('H').mean()
    elif interval == 'Daily':
        resampled_data = data.resample('D').mean()
    elif interval == 'Monthly':
        resampled_data = data.resample('M').mean()
    else:
        resampled_data = data.resample('Y').mean()

    plt.figure(figsize=(12, 6))
    plt.plot(resampled_data.index, resampled_data['Close'], label=f'{interval} Close Price')
    plt.axvline(x=data.index[-1], color='r', linestyle='--', label='Last Data Point')
    plt.plot(data.index[-1], next_day_close, 'go', markersize=8, label='Next Day Prediction')

    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title('Close Price with Predicted Values')

    plt.legend()
    plt.grid(True)
    plt.show()

# Connect the filter dropdown to the update function
widgets.interactive(update_plot, interval=interval_dropdown)


interactive(children=(Dropdown(description='Interval:', index=1, options=('Hourly', 'Daily', 'Monthly', 'Yearl…

In [237]:
data.tail()

,Close,Open,High,Low,Vol(millions),NextDayClose,NextDayPrice,NextDayPercentageChange
Date,,,,,,,,
2023-07-03,3.86,3.51,3.86,3.55,74.96,3.48,-0.38,-9.844560
2023-07-04,3.48,3.86,4.20,3.48,105.87,3.63,0.15,4.310345
2023-07-05,3.63,3.48,3.70,3.48,89.23,3.76,0.13,3.581267
2023-07-06,3.76,3.63,3.83,3.65,49.18,4.13,0.37,9.840426
2023-07-07,4.13,3.76,4.13,3.82,79.65,NaN,NaN,NaN
